In [ ]:
# Leave for imports

Hope you're excited for the next Arctic Fox step! In this sandbox, we'll show you Automations working together and chaining one after another. Also, we'll start referring to Automations as autos, auto singular, for short. 

# Chained Automations

To illustrate, we'll use a simple image processing example. We'll show the image after each step using the Visualize auto. In our scenario, we have an image we want to use as a background for slides, but the image is too bright and too busy. Let's take a look!

First, let's load the image with the Data auto. The Data auto can be used with a wide variety of data / file formats, for now, we'll stick with images. The Data auto will also serve as the first step in a chain of autos. 

In [ ]:
#[Data Waves.jpg]

#[Visualize]

As you can see, the image is pretty bright and busy. Let's make the image darker with the Darken auto. You'll notice that Darken connects to the chain and uses the variable from the Data auto. Additionally, Visualize will see that Darken is now the most recent auto in the chain and will show Darken's output, instead of Data's.    

In [ ]:
#[Darken 20%]

#[Visualize]

That's a little better, but the image is still too busy. Let's use the Blur auto to, as you guessed, blur the image. Again, Blur will connect to the chain and use the variable from Darken instead of Data.

In [ ]:
#[Blur 15]

#[Visualize]

That looks like a good slide background! It's got a bit happening, but not too much! Now, let's save the result with the SaveImage auto. Again, this auto will connect to the chain and use the most recent auto in the chain. 

In [ ]:
#[SaveImage]

There you have it! Let's go to the next sandbox which will build on this image processing example!